In [1]:
# coding=utf-8
import os
import json
import re

import pandas as pd

### 加载user.csv

In [2]:
user_path = '../dataset/orign_data/dataset1/user.csv'
df_user = pd.read_csv(user_path)
df_user.columns = ['rating', 'user_name',
                   'comment_time', 'user_id', 'movie_name', 'genres']
df_user.head()

,rating,user_name,comment_time,user_id,movie_name,genres
0,2,身似,2018-01-05 15:05:06,1,心雨花露,爱情
1,4,有意识的贱民,2018-01-05 15:05:06,3,战争的恐怖,战争
2,2,亿万露电,2018-01-05 15:05:06,4,豪勇七蛟龙,战争
3,2,Marni,2018-01-05 15:05:06,5,无序之主,犯罪
4,4,马西嘻嘻嘻,2018-01-05 15:05:06,6,时装店风波,同性


### 加载movie.csv

In [3]:
movie_path = '../dataset/orign_data/dataset1/movie.csv'
df_movie = pd.read_csv(movie_path)
df_movie.columns = ['genres', 'actors', 'contries',
                    'director', 'special', 'rating', 'movie_name']
df_movie.head()

,genres,actors,contries,director,special,rating,movie_name
0,剧情,徐峥|王传君|周一围|谭卓|章宇,中国大陆,文牧野,经典,8.9,我不是药神
1,剧情,冯小刚|许晴|张涵予|刘桦|李易峰,中国大陆,管虎,经典,7.8,老炮儿
2,剧情,王宝强|刘昊然|肖央|刘承羽|尚语贤,中国大陆,陈思诚,经典,6.7,唐人街探案2
3,剧情,任素汐|大力|刘帅良|裴魁山|阿如那,中国大陆,周申|刘露,经典,8.3,驴得水
4,剧情,徐峥|王宝强|李曼|李小璐|左小青,中国大陆,叶伟民,经典,7.5,人在囧途


### 加载node

In [4]:
node_path = '../dataset/orign_data/dataset2/nodes.csv'
df_node = pd.read_csv(node_path, header=None)
df_node.columns = ['user_id']
df_node.head()

,user_id
0,154907
1,1
2,2
3,3
4,4


### 加载edges

In [5]:
edges_path = '../dataset/orign_data/dataset2/edges.csv'
df_edges = pd.read_csv(edges_path, header=None)
df_edges.columns = ['user_id', 'focus_id']
df_edges.head()

,user_id,focus_id
0,154907,1
1,154907,2
2,154907,3
3,154907,4
4,154907,5


### 处理comment

In [6]:
comment_path = '../dataset/orign_data/DMSC.csv'
df_comment=pd.read_csv(comment_path)
df_comment.columns = ['ID','movie_name_en','movie_name_cn','crawl_date','number','user_name','date','star','comment','like']
df_comment.head()

,ID,movie_name_en,movie_name_cn,crawl_date,number,user_name,date,star,comment,like
0,0,Avengers Age of Ultron,复仇者联盟2,2017-01-22,1,然潘,2015-05-13,3,连奥创都知道整容要去韩国。,2404
1,1,Avengers Age of Ultron,复仇者联盟2,2017-01-22,2,更深的白色,2015-04-24,2,非常失望，剧本完全敷衍了事，主线剧情没突破大家可以理解，可所有的人物都缺乏动机，正邪之间、...,1231
2,2,Avengers Age of Ultron,复仇者联盟2,2017-01-22,3,有意识的贱民,2015-04-26,2,2015年度最失望作品。以为面面俱到，实则画蛇添足；以为主题深刻，实则老调重弹；以为推陈出...,1052
3,3,Avengers Age of Ultron,复仇者联盟2,2017-01-22,4,不老的李大爷耶,2015-04-23,4,《铁人2》中勾引钢铁侠，《妇联1》中勾引鹰眼，《美队2》中勾引美国队长，在《妇联2》中终于...,1045
4,4,Avengers Age of Ultron,复仇者联盟2,2017-01-22,5,ZephyrO,2015-04-22,2,虽然从头打到尾，但是真的很无聊啊。,723


### 处理movie.json

In [7]:
result = []
movie_json_path='../dataset/orign_data/movie.json'
with open(movie_json_path, 'r') as f:
    data_list = f.readlines()
    for each in data_list:
        each = json.loads(each)
        directors_list = each.get('directors', [])
        first_director = directors_list[0].get('name') if directors_list else None
        directors = '|'.join(map(lambda x: x.get('name'), directors_list))
        genres_list = each.get('genres', [])
        first_genre = genres_list[0] if genres_list else None
        genres = '|'.join(genres_list)
        rating_avg = each.get('rating', {}).get('average')
        rating_people = each.get('rating', {}).get('rating_people')
        stars = each.get('rating', {}).get('stars')
        if isinstance(stars, list) and len(stars) == 5:
            star_5, star_4, star_3, star_2, star_1 = stars
        else:
            star_5, star_4, star_3, star_2, star_1 = [None]*5
        publish_date_list = each.get('pubdate', [])
        publish_date = '|'.join(publish_date_list)
        if publish_date and re.search('(\d{4}-?\d*-?\d*)', publish_date):
            publish_date = re.search(
                '(\d{4}-?\d*-?\d*)', publish_date).group(1)
        year = each.get('year')
        if not re.search('(\d{4})', year):
            year = None
        else:
            year = re.search('(\d{4})', year).group(1)
        contries_list = each.get('countries',[])
        first_contry = contries_list[0] if contries_list else None
        contries = '|'.join(contries_list)
        movie_id = each.get('lens_id')
        actors_list = each.get('casts',[])
        first_actor = actors_list[0].get('name') if actors_list else None
        actors = '|'.join(map(lambda x: x.get('name'), actors_list))
        movie_name = re.search('(\w+)', each.get('title')).group(1)
        summary = each.get('summary')
        languages_list = each.get('languages')
        first_language = languages_list[0] if languages_list else None
        languages = '|'.join(languages_list)
        writers_list = each.get('writers',[])
        first_writer = writers_list[0].get('name') if writers_list else None
        writers = '|'.join(map(lambda x: x.get('name'), writers_list))
        data = {
            'directors': directors,
            'first_director':first_director,
            'genres': genres,
            'fist_genre':first_genre,
            'rating_avg': rating_avg,
            'rating_people': rating_people,
            'star_5': star_5,
            'star_4': star_4,
            'star_3': star_3,
            'star_2': star_2,
            'star_1': star_1,
            'publish_date': publish_date,
            'publish_year': year,
            'contries': contries,
            'first_contry':first_contry,
            'movie_id': movie_id,
            'actors': actors,
            'first_actor':first_actor,
            'movie_name': movie_name,
            'summary': summary,
            'languages': languages,
            'first_language':first_language,
            'writers': writers,
            'first_writer':first_writer
        }
        result.append(data)
df_movie_json = pd.DataFrame(result)
df_movie_json.head()

,directors,first_director,genres,fist_genre,rating_avg,rating_people,star_5,star_4,star_3,star_2,...,first_contry,movie_id,actors,first_actor,movie_name,summary,languages,first_language,writers,first_writer
0,马丁·里特,马丁·里特,剧情|动作|西部,剧情,8.0,145,24.0,53.8,19.2,2.9,...,美国,5435,保罗·纽曼|弗雷德里克·马奇|理查德·布恩|黛安妮·赛琳托|卡梅隆·米切尔,保罗·纽曼,野狼,约翰·罗塞尔自幼是老罗塞尔先生从战俘中带回来并抚养他长大的，但是他生性豪放不羁，又回到土著人...,英语,英语,埃尔莫·伦纳德|Harriet Frank Jr.|欧文·拉文奇,埃尔莫·伦纳德
1,约翰·班德汉姆,约翰·班德汉姆,剧情|惊悚|犯罪,剧情,6.5,1774,7.6,25.2,54.7,10.4,...,美国,89,约翰尼·德普|Courtney Chase|查尔斯·达顿|克里斯托弗·沃肯|罗马·玛菲娅|彼...,约翰尼·德普,千钧一发,韦斯顿（约翰尼·德普JohnnyDepp饰）的前妻去世了，带着小女儿琳恩（CourtneyC...,英语,英语,Patrick Sheane Duncan,Patrick Sheane Duncan
2,今石洋之,今石洋之,喜剧|动作|科幻|动画|冒险,喜剧,8.6,3458,48.2,38.0,12.1,1.0,...,日本,62331,山口胜平|本田贵子|水谷优子|飞田展男|高木渉|岩田光央|梁田清之,山口胜平,落叶,在未来的某一天，一对男女Retro（山口胜平配音）和Pandy（本田贵子配音）在一片荒野中醒...,日语,日语,本田武市,本田武市
3,比班·基德龙,比班·基德龙,喜剧|爱情,喜剧,7.3,61303,13.7,41.1,39.8,4.8,...,英国,8969,蕾妮·泽尔维格|科林·费尔斯|休·格兰特|吉姆·布劳德本特|杰茜达·芭瑞特,蕾妮·泽尔维格,BJ单身日记2,布里奇特•琼斯（蕾妮•齐维格饰）自从与律师马克•达西（科林•菲尔斯饰）坠入爱河后，心想着与男...,英语|德语,英语,亚当·布鲁克斯|安德鲁·戴维斯|海伦·费尔丁|理查德·柯蒂斯|海伦·费尔丁,亚当·布鲁克斯
4,Tom Gianas|Ross Shuman,Tom Gianas,喜剧|动画|冒险,喜剧,6.6,2920,9.7,27.3,49.0,11.5,...,美国,150278,尼克·斯旺森|米拉·库尼斯|鲍勃·奥登科克|T·J·米勒|罗伯·里格尔|苏珊·萨兰登|丹尼·...,尼克·斯旺森,地狱归来,三个打小一块儿长大的好伙伴雷米（斯旺森配音）、奥吉（T.J.米勒配音）以及科特（里格尔配音）...,英语,英语,Tom Gianas|Hugh Sterbakov|Zeb Wells,Tom Gianas


In [8]:
df_movie_json.columns

Index(['directors', 'first_director', 'genres', 'fist_genre', 'rating_avg',
       'rating_people', 'star_5', 'star_4', 'star_3', 'star_2', 'star_1',
       'publish_date', 'publish_year', 'contries', 'first_contry', 'movie_id',
       'actors', 'first_actor', 'movie_name', 'summary', 'languages',
       'first_language', 'writers', 'first_writer'],
      dtype='object')

In [9]:
df_movie_json = df_movie_json[['movie_id', 'movie_name', 'genres', 'fist_genre', 'directors', 'first_director', 'actors', 'first_actor',
               'contries', 'first_contry', 'languages','first_language', 'writers', 'first_writer', 'publish_date', 'publish_year',
               'summary', 'rating_people', 'star_5', 'star_4', 'star_3', 'star_2', 'star_1','rating_avg']].sort_values('movie_id')
df_movie_json.head()
df_movie_json.to_csv('../dataset/cleaned_data/movie.csv',index=False)

In [10]:
df_user_cleaned = df_comment[['user_name']].drop_duplicates(ignore_index=True).reset_index()
df_user_cleaned['index']=df_user_cleaned['index']+1
df_user_cleaned.columns = ['user_id','user_name']
df_user_cleaned.to_csv('../dataset/cleaned_data/user.csv',index=False)
df_user_cleaned.head()

,user_id,user_name
0,1,然潘
1,2,更深的白色
2,3,有意识的贱民
3,4,不老的李大爷耶
4,5,ZephyrO


In [11]:
df_comment_clean = pd.merge(df_comment,df_user_cleaned)
df_comment_clean = pd.merge(df_comment_clean,df_movie_json[['movie_id','movie_name']],left_on='movie_name_cn',right_on='movie_name')
df_comment_clean.head()

,ID,movie_name_en,movie_name_cn,crawl_date,number,user_name,date,star,comment,like,user_id,movie_id,movie_name
0,0,Avengers Age of Ultron,复仇者联盟2,2017-01-22,1,然潘,2015-05-13,3,连奥创都知道整容要去韩国。,2404,1,122892,复仇者联盟2
1,1,Avengers Age of Ultron,复仇者联盟2,2017-01-22,2,更深的白色,2015-04-24,2,非常失望，剧本完全敷衍了事，主线剧情没突破大家可以理解，可所有的人物都缺乏动机，正邪之间、...,1231,2,122892,复仇者联盟2
2,2,Avengers Age of Ultron,复仇者联盟2,2017-01-22,3,有意识的贱民,2015-04-26,2,2015年度最失望作品。以为面面俱到，实则画蛇添足；以为主题深刻，实则老调重弹；以为推陈出...,1052,3,122892,复仇者联盟2
3,3,Avengers Age of Ultron,复仇者联盟2,2017-01-22,4,不老的李大爷耶,2015-04-23,4,《铁人2》中勾引钢铁侠，《妇联1》中勾引鹰眼，《美队2》中勾引美国队长，在《妇联2》中终于...,1045,4,122892,复仇者联盟2
4,4,Avengers Age of Ultron,复仇者联盟2,2017-01-22,5,ZephyrO,2015-04-22,2,虽然从头打到尾，但是真的很无聊啊。,723,5,122892,复仇者联盟2


In [12]:
df_comment_clean[['user_id', 'user_name', 'movie_id', 'movie_name_en', 'movie_name_cn', 'date',
                  'comment', 'like', 'star']].to_csv('../dataset/cleaned_data/comment.csv', index=False)

In [ ]:
df_rating = df_comment_clean[['user_id','movie_id','star']]
df_rating.to_csv('../dataset/cleaned_data/rating.csv',index)